In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import keyword
import torch
from torch.utils.tensorboard import SummaryWriter
from framework.policies.ppo_no_scaling_rec_global_critic import NNN
import os

In [30]:
from framework.policies.ppo_shared_use_future import NNN
path = os.path.abspath("experiments")+"/ppo_shared_use_future-full_communication_2-big_net_bigger_bottleneck/saved_models/agent_0"

model = NNN(32, 2, 50, 128)
model.load_state_dict(torch.load(path), strict=False)

model.cuda()

# def get_repr(model, c, goal_color):
#     x = torch.zeros(32)
#     comm = torch.zeros(10)
#     comm[c] = 1
#     x[-10:] = comm
    
#     for i in range(4):
#         x[4+i*3] = (i+0.1)/5
#         x[5+i*3] = (i+0.1)/5
#         x[6+i*3] = colors[i]
    
#     x[2] = goal_color
#     x[3] = goal_color
#     x = x[None,None,:].cuda()
#     model.init_hidden(1)
#     return model.get_hidden(x).cpu().detach()[0][0]
def get_repr_x(model, x):
    x = x[None,None,:].cuda()
    model.init_hidden(1)
    return model.get_hidden(x).cpu().detach()[0][0]

def get_repr_C(model, c):
    x = torch.zeros(32)-1
    comm = torch.zeros(10)
    comm[c] = 1
    x[-10:] = comm
    
    x = x[None,None,:].cuda()
    model.init_hidden(1)
    return model.get_hidden(x).cpu().detach()[0][0]

In [31]:
vocab = "*ABCDEFGHIJKLMNOPQRSTUVX"[0:10]
colors = [1, 0, 0.5, 0.75]
color_names = ["red","green","yellow","blue"]

In [32]:
names = []
embedding = []
for i, v in enumerate(vocab):
    x = torch.zeros(32)-1
    comm = torch.zeros(10)
    comm[i] = 1
    x[-10:] = comm
    embedding.append(get_repr_x(model, x))
    names.append(vocab[i])

for color, color_name in zip(colors, color_names):
    x = torch.zeros(32)-1
    x[2] = x[3] = color
    embedding.append(get_repr_x(model, x))
    names.append("goal_"+color_name)

for i in range(4):
    for j, (color, color_name) in enumerate(zip(colors, color_names)):
        x = torch.zeros(32)-1
        x[4+i*3] = x[5+i*3] = 1
        x[6+i*3] = color
        embedding.append(get_repr_x(model, x))
        names.append(f"landmark_{i}_"+color_name)

for i, v in enumerate(vocab):
    x = torch.zeros(32)-1
    comm = torch.zeros(10)
    comm[i] = 1
    x[-10:] = comm
    for color, color_name in zip(colors, color_names):
        x[2] = x[3] = color
        embedding.append(get_repr_x(model, x))
        names.append(vocab[i]+"_goal_"+color_name)

for k, v in enumerate(vocab):
    for i in range(4):
        x = torch.zeros(32)-1
        comm = torch.zeros(10)
        comm[k] = 1
        x[-10:] = comm
        for _, (color, color_name) in enumerate(zip(colors, color_names)):
            x[4+i*3] = x[5+i*3] = 1
            x[6+i*3] = color
            embedding.append(get_repr_x(model, x))
            names.append(f"{vocab[k]}_landmark_{i}_"+color_name)

for i in range(4):
    for color_goal, color_goal_name in zip(colors, color_names):
        x = torch.zeros(32)-1
        x[2] = x[3] = color_goal
        for j, (color, color_name) in enumerate(zip(colors, color_names)):
            x[4+i*3] = x[5+i*3] = 1
            x[6+i*3] = color
            embedding.append(get_repr_x(model, x))
            names.append(f"goal_{color_goal_name}_landmark_{i}_"+color_name)

embedding = torch.vstack(embedding)

In [33]:


writer = SummaryWriter("simple6")
writer.add_embedding(embedding, metadata=names)

In [34]:
embedding

tensor([[20.3228,  0.8163,  6.0628,  ...,  0.0000, 21.3462,  2.5827],
        [22.8575,  0.7777,  6.2456,  ...,  0.0000, 23.5095,  3.3312],
        [21.7452,  0.8898,  5.8299,  ...,  0.0000, 21.8959,  2.6577],
        ...,
        [27.8619,  0.0000,  6.7523,  ...,  0.0000, 33.9109,  6.2006],
        [27.4077,  0.0000,  6.8302,  ...,  0.0000, 33.2574,  6.1668],
        [27.1135,  0.0000,  6.8692,  ...,  0.0000, 32.8486,  6.1380]])